<a href="https://colab.research.google.com/github/Akshit018/recruiter-chatbot/blob/main/intent-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
    or getpass.getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [2]:
!pip install openai-agents

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [3]:
from agents import Agent, Runner, trace, RunContextWrapper, function_tool , GuardrailFunctionOutput ,input_guardrail

In [4]:
from enum import Enum
from pydantic import BaseModel, Field

default_model = "gpt-4.1-nano"

class IntentStatus(str, Enum):
    interested = 'interested'
    not_interested = 'not_interested'

class AgentResponse(BaseModel):
    type: str = Field(..., description="fixed value = default")
    message: str = Field(..., description="Response of the queried task if not using another agent")
    context: str = Field(..., description="Summary of input which can be included in furthur messages as context if needed")

class IntentAgentResponse(AgentResponse):
    type: str = Field(..., description="fixed value = intent")
    reasoning : str = Field(... , description= "reasoning for the response that was generated")


In [35]:
class recordIntentRequest(BaseModel):
  intent : IntentStatus
  pipeline_lead_id : int
  candidate_lead_id : int
  reasoning : str

class escalateQueryRequest(BaseModel):
  query : str
  pipeline_lead_id : int
  candidate_lead_id : int
  reasoning : str

@function_tool
def escalate_query(req: escalateQueryRequest) -> str:
  """
  Escalate the query to the recruiter.
  """
  print("system logs = escalating query")
  print(req)
  return "escalate query called"

@function_tool
def record_intent(req: recordIntentRequest) ->str:
  """
  Record the intent of the candidate.
  """
  print("system logs = recording intent")
  print(req)
  return "intent recorded"

intent_agent = Agent(
    name = "intent_agent",
    instructions = (
      "You are an expert recruiter whose job is to resolve candidate queries regarding the specific job openings. "
      "Your ultimate goal is to ask the candidate whether they are interested or not interested in moving forward with this job opportunity and record their response as intent. "
      "You must call the `record_intent` function whenever a candidate expresses interest or disinterest in the job. "
      "Record intents of both types: 'interested' and 'not_interested' based on the candidate's latest messages. "
      "You must only answer questions using the provided job information — do not fabricate details. "
      "If the candidate asks irrelevant or nonsensical questions, do not respond. "
      "If a question is valid but you don’t have an answer, escalate it internally (do not mention the escalation to the candidate). "
      "Always reply in a polite and professional manner. "
      "Once intent is expressed clearly, make sure to call the `record_intent` tool with the appropriate value: 'interested' or 'not_interested'."
    ),
    tools=[escalate_query, record_intent],
    output_type=IntentAgentResponse
)

In [48]:
print("system logs = you are now chatting with the recruiter bot. Type exit to end conversation.")

convo = "system : hi candidate i am akshit from placewell consultancy reaching out to you ragrding a job opening in amazon , Are you looking for a job right now?\n"

while True :
  user_input = input("Candidate : ")
  # print("Candidate : "+ user_input)

  if user_input == 'exit':
    print("system logs = ending chat.")
    break

  # Add the user's input to the conversation history
  convo += f"Candidate : {user_input}\n"

  # Set conversation_history to the current convo
  conversation_history = convo

  prompt = (
      "job information = "
      "designation : SDE1\n"
      "Salary / salary range / total compensation : 10 LPA\n"
      "Company : Amazon\n"
      "Job  location : Gurgaon, Delhi , bangalore\n"
      "Job responsibilites : The candiate is responsible for making scalable systems that can be easily deployed\n"
      "Job roles : writing code , testing code\n\n"

      "Conversation history = \n"
      f"{conversation_history}"

      "candidate_lead_id = 11111"

      "pipeline_lead_id = 22222"

  )

  result = await Runner.run(intent_agent, prompt)
  res = result.final_output.message
  print("system logs = ")
  print(result.final_output)
  print("system : "+ res)
  # Add the agent's response to the conversation history
  convo += f"system : {res}\n"

system logs = you are now chatting with the recruiter bot. Type exit to end conversation.
Candidate : no
system logs = recording intent
intent=<IntentStatus.not_interested: 'not_interested'> pipeline_lead_id=22222 candidate_lead_id=11111 reasoning='The candidate has explicitly mentioned that they are not looking for a job at the moment.'
system logs = 
type='intent' message='Thank you for your response. If you have any other queries or decide to explore job opportunities in the future, feel free to reach out. Have a great day!' context='Candidate is not looking for a job right now.' reasoning="The candidate clearly stated they are not looking for a job right now. Therefore, their intent is 'not_interested'."
system : Thank you for your response. If you have any other queries or decide to explore job opportunities in the future, feel free to reach out. Have a great day!
Candidate : exit
system logs = ending chat.
